---
title: "Transit Mode"
format:
  html:
    toc: true
    css: styles.css
    echo: true
    code-fold: true
---


In [ ]:
# | echo: false
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import toml
from IPython.display import HTML
import plotly.io as pio

pio.renderers.default = "notebook"

CONFIG = toml.load("config.toml")

In [ ]:
obs2023 = pd.read_parquet("../../../data/obs2023.parquet")
obs2023

In [ ]:
# | code-summary: "Data"
# Categorize based on HHTS categories
hhts_ages = {
    "Under 5": 4,
    "5-15": 15,
    "16-17": 17,
    "17-18": 18,
    "25-34": 34,
    "35-44": 44,
    "45-54": 54,
    "55-64": 64,
    "65-74": 74,
    "75-84": 84,
    "85 or Older": np.inf,
}

age = pd.DataFrame(
    {
        "age": (obs2023.DATE_COMPLETED.dt.year - obs2023.YEAR_BORN),
        "age_category": pd.cut(
            # Calculate age based on survey date and respondents birth year
            x=(obs2023.DATE_COMPLETED.dt.year - obs2023.YEAR_BORN),
            bins=[0] + list(hhts_ages.values()),
            labels=list(hhts_ages.keys()),
        ),
        "unlinked_weight": obs2023.UNLINKED_WGHT_FCTR,
        "linked_weight": obs2023.LINKED_WGHT_FCTR,
    }
)

In [ ]:
# | code-summary: "Table"

unweighted = age.groupby("age_category", observed=True).size()
unlinked = age.groupby("age_category", observed=True).unlinked_weight.sum()
linked = age.groupby("age_category", observed=True).linked_weight.sum()
total_valid = age.dropna().size
total = age.size

summary = pd.DataFrame(
    {
        ("unweighted", "count"): unweighted,
        ("unweighted", "percent"): unweighted / unweighted.sum(),
        ("unlinked_weight", "count"): unlinked,
        ("unlinked_weight", "percent"): unlinked / unlinked.sum(),
        ("linked_weight", "count"): linked,
        ("linked_weight", "percent"): linked / linked.sum(),
    }
)

table = pd.concat(
    [
        summary,
        pd.DataFrame(
            [
                {
                    "age_category": "Total valid",
                    ("unweighted", "count"): total_valid,
                    ("unweighted", "percent"): total_valid / total,
                },
                {"age_category": "Total", ("unweighted", "count"): total},
            ],
            index=["Total valid", "Total"],
        ).set_index("age_category"),
    ]
).to_html(
    formatters={
        ("unweighted", "count"): "{:,.0f}".format,
        ("unweighted", "percent"): "{:,.2%}".format,
        ("unlinked_weight", "count"): "{:,.0f}".format,
        ("unlinked_weight", "percent"): "{:,.2%}".format,
        ("linked_weight", "count"): "{:,.0f}".format,
        ("linked_weight", "percent"): "{:,.2%}".format,
    }
)

HTML(table)